In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import time
from scipy.signal import butter, filtfilt, resample

# ======================
# MODEL DEFINITION
# ======================

WINDOW_SEC = 8  # Changed from 7 to 8 to match model weights
MODEL_FS = 100  # trained at 100 Hz
STEP_SEC = 2

WINDOW_SAMPLES = WINDOW_SEC * MODEL_FS

class BPModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv1d(2, 32, 7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(32, 64, 5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, 5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.regressor = nn.Sequential(
            nn.Linear(128 * (WINDOW_SAMPLES // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.regressor(x)

# ======================
# LOAD MODEL
# ======================

model = BPModel()
model.load_state_dict(torch.load(r"D:\BTP\BTP4\realtime_rppg_webapp\bp_model_ppg.pth", map_location="cpu"))
model.eval()

# ======================
# BANDPASS FILTER (0.7–3 Hz)
# ======================

def bandpass(signal, fs):
    b, a = butter(2, [0.7/(fs/2), 3/(fs/2)], btype='band')
    return filtfilt(b, a, signal)

# ======================
# CAMERA SETUP
# ======================

cap = cv2.VideoCapture(0)
buffer = []
timestamps = []

print("Place your finger on the camera with flash ON.")
print("Collecting signal...")

last_prediction_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Extract green channel average
    green = np.mean(frame[:,:,1])
    buffer.append(green)
    timestamps.append(time.time())

    # Keep last 15 seconds only
    if len(buffer) > 500:
        buffer.pop(0)
        timestamps.pop(0)

    current_time = time.time()

    # Predict every 2 seconds
    if current_time - last_prediction_time >= STEP_SEC:

        duration = timestamps[-1] - timestamps[0]

        if duration >= WINDOW_SEC:

            signal = np.array(buffer)

            # Estimate sampling rate
            fs_cam = len(signal) / duration

            # Filter
            filtered = bandpass(signal, fs_cam)

            # Resample to 100 Hz
            resampled = resample(filtered, WINDOW_SAMPLES)

            # Normalize
            resampled = (resampled - np.mean(resampled)) / np.std(resampled)

            # First derivative
            d1 = np.gradient(resampled)

            # Stack channels
            input_signal = np.vstack([resampled, d1])
            input_signal = torch.tensor(input_signal, dtype=torch.float32).unsqueeze(0)

            # Predict
            with torch.no_grad():
                pred = model(input_signal).numpy()[0]

            sbp, dbp = pred

            print(f"Predicted SBP: {sbp:.1f} mmHg | DBP: {dbp:.1f} mmHg")

            last_prediction_time = current_time

    cv2.imshow("Camera", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Place your finger on the camera with flash ON.
Predicted SBP: 122.5 mmHg | DBP: 75.0 mmHg
Predicted SBP: 138.4 mmHg | DBP: 85.3 mmHg
Predicted SBP: 124.7 mmHg | DBP: 76.5 mmHg
Predicted SBP: 128.5 mmHg | DBP: 79.1 mmHg
Predicted SBP: 123.9 mmHg | DBP: 76.0 mmHg
Predicted SBP: 135.6 mmHg | DBP: 83.2 mmHg
Predicted SBP: 140.7 mmHg | DBP: 86.4 mmHg
Predicted SBP: 124.4 mmHg | DBP: 76.3 mmHg
Predicted SBP: 127.9 mmHg | DBP: 78.1 mmHg
Predicted SBP: 145.8 mmHg | DBP: 89.3 mmHg
Predicted SBP: 123.3 mmHg | DBP: 75.2 mmHg
Predicted SBP: 130.6 mmHg | DBP: 81.2 mmHg
Predicted SBP: 132.7 mmHg | DBP: 81.8 mmHg
Predicted SBP: 148.9 mmHg | DBP: 91.5 mmHg
Predicted SBP: 141.5 mmHg | DBP: 87.7 mmHg
Predicted SBP: 124.7 mmHg | DBP: 76.7 mmHg
Predicted SBP: 129.9 mmHg | DBP: 80.5 mmHg
Predicted SBP: 147.3 mmHg | DBP: 92.6 mmHg
Predicted SBP: 144.9 mmHg | DBP: 89.3 mmHg
Predicted SBP: 137.0 mmHg | DBP: 84.8 mmHg
Predicted SBP: 136.0 mmHg | DBP: 84.1 mmHg
Predicted SBP: 149.2 mmHg | DBP: 92.4 mmHg
Predict

KeyboardInterrupt: 

: 